## Load WRI Power Plant data from 2017 dataset (see https://github.com/wri/global-power-plant-database) for original source

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have a local copy in an s3 bucket here: s3://ocp-odh-os-demo-s3/data/WRI-upload/wri-pp.tsv
### To tidy the data we factor into two tables:

* **wri_plants** (all the fixed data about each plant)
* **wri_annual_gwh** (per plant/per year annual generation in GWh)

### The next step is to annotate **wri_annual_gwh** with OS-C Factor metadata

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

### Load python modules if starting from a minimal notebook

In [1]:
# %pip install python-dotenv
# %pip install boto3
# %pip install pandas

### Load Credentials

In [2]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

### Load WRI data file using pandas *read_csv*

In [3]:
import boto3
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_ENDPOINT'],
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
)

obj = s3.get_object(
    Bucket=os.environ['S3_BUCKET'],
    Key='data/WRI-upload/wri-pp.tsv')

In [4]:
import pandas as pd

wri_file = pd.read_csv(obj['Body'], delimiter='\t')

display(wri_file.columns)

Index(['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'generation_gwh_2017',
       'estimated_generation_gwh'],
      dtype='object')

### Melt the generation data into a more tidy format, dropping NA values

In [5]:
wri_plants = wri_file[['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data', 'estimated_generation_gwh']]

wri_id_vars =['gppd_idnr']
wri_value_vars = ['generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'generation_gwh_2017']
wri_annual_gwh = wri_file.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='generation_gwh')
wri_annual_gwh['year'] = wri_annual_gwh['year'].apply(lambda x: int(x.split('_')[-1]))
wri_annual_gwh.dropna(subset=['generation_gwh'],inplace=True)

display(wri_annual_gwh)

,gppd_idnr,year,generation_gwh
2,USA0057062,2013,1.519000
3,USA0057052,2013,1.821000
4,USA0007485,2013,6.346000
5,USA0054422,2013,31.560000
6,USA0054305,2013,18.865000
...,...,...,...
149470,AUS0000078,2017,13.434722
149473,AUS0000073,2017,35.649167
149475,AUS0000347,2017,13.954167
149477,AUS0000268,2017,214.876389


In [6]:
display(wri_plants)

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,estimated_generation_gwh
0,FJI,Fiji,Waiyevo,WRI1061477,3.00,-16.7844,-179.9777,Oil,NaN,NaN,NaN,NaN,Fiji Electricity Authority,Fiji Electricity Authority,http://www.energyandinfrastructure.com/section...,WRI,NaN,NaN,NaN
1,RUS,Russia,Egvenkinotskaya TPP,WRI1003701,34.00,66.4013,-179.1338,Gas,NaN,NaN,NaN,NaN,"JSC ""Chukotenergo""",EnergyBase (RUS),https://energybase.ru/power-plant/Egvekinotska...,EnergyBase (RUS),NaN,NaN,179.062963
2,USA,United States of America,Gambell,USA0057062,1.60,63.7771,-171.7124,Oil,NaN,NaN,NaN,2006.000000,Alaska Village Elec Coop Inc,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,12671,2017.0,NaN
3,USA,United States of America,Savoonga,USA0057052,1.70,63.6953,-170.4757,Oil,NaN,NaN,NaN,2008.000000,Alaska Village Elec Coop Inc,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,12698,2017.0,NaN
4,USA,United States of America,NSB Point Hope Utility,USA0007485,3.10,68.3484,-166.7372,Oil,NaN,NaN,NaN,2004.903226,North Slope Borough Power & Light,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,28167,2017.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29905,FJI,Fiji,Wainikasaou,WRI1061468,6.00,-17.8331,178.0270,Hydro,NaN,NaN,NaN,2004.000000,Fiji Electricity Authority,Fiji Electricity Authority,http://www.energyandinfrastructure.com/section...,WRI,1057397,NaN,NaN
29906,FJI,Fiji,Wailoa,WRI1061469,160.00,-17.7569,178.0498,Hydro,NaN,NaN,NaN,1983.000000,Fiji Electricity Authority,Fiji Electricity Authority,http://www.energyandinfrastructure.com/section...,WRI,1017016,NaN,NaN
29907,FJI,Fiji,Kinoya,WRI1061458,48.00,-18.1127,178.4799,Oil,NaN,NaN,NaN,2015.000000,Fiji Electricity Authority,Fiji Electricity Authority,http://www.energyandinfrastructure.com/section...,WRI,1017003,NaN,NaN
29908,FJI,Fiji,Levuka Power Station,WRI1061473,2.98,-17.6845,178.8359,Oil,NaN,NaN,NaN,NaN,Fiji Electricity Authority,Fiji Electricity Authority,http://www.energyandinfrastructure.com/section...,WRI,1017007,NaN,NaN
